In [3]:
import math
import numpy as np
import pandas as pd
from datetime import timedelta
from datetime import datetime
from dateutil.parser import parse
import plotly.io as pio
import plotly.express as px
import plotly.graph_objs as go
from plotly.subplots import make_subplots
print(pio.renderers)
pio.renderers.default = "notebook"

Renderers configuration
-----------------------
    Default renderer: 'vscode'
    Available renderers:
        ['plotly_mimetype', 'jupyterlab', 'nteract', 'vscode',
         'notebook', 'notebook_connected', 'kaggle', 'azure', 'colab',
         'cocalc', 'databricks', 'json', 'png', 'jpeg', 'jpg', 'svg',
         'pdf', 'browser', 'firefox', 'chrome', 'chromium', 'iframe',
         'iframe_connected', 'sphinx_gallery']



In [20]:
path = "/home/waris/Github/tupras-analysis/new-alarm/stats-one-month.csv"
df = pd.read_csv("stats-one-month.csv", low_memory=False)
df["StartTime"] = df["StartTime"].apply(lambda d: parse(d))
df["EndTime"] = df["EndTime"].apply(lambda d: parse(d))
df["Time"] = df["Time"].apply(lambda d: datetime.strptime(d,"%H:%M").time())
# for col in df.columns:
#     print(col, type(df[col][0]))
# print("===============")
# df.info()
df.head(20)

In [17]:
# df.describe()

,Quality,Mask,NewState,Status,TimeDelta,Year,Month,MonthDay,Hour,Minute
count,284750.000000,284750.000000,284750.000000,284750.000000,284750.000000,284750.0,284750.0,284750.000000,284750.000000,284750.000000
mean,37.271263,199.261517,3.000471,0.964158,347.286160,2019.0,3.0,14.422005,11.323326,29.279856
std,75.940474,4.231012,0.036528,0.185896,5354.781415,0.0,0.0,5.298439,7.128231,17.314135
min,0.000000,128.000000,3.000000,0.000000,0.000000,2019.0,3.0,6.000000,0.000000,0.000000
25%,0.000000,201.000000,3.000000,1.000000,16.000000,2019.0,3.0,10.000000,5.000000,14.000000
50%,0.000000,201.000000,3.000000,1.000000,29.000000,2019.0,3.0,14.000000,11.000000,29.000000
75%,0.000000,201.000000,3.000000,1.000000,44.000000,2019.0,3.0,19.000000,18.000000,44.000000
max,192.000000,249.000000,7.000000,1.000000,482009.000000,2019.0,3.0,23.000000,23.000000,59.000000


# StartTime 

The following graph shows all the alarms triggered in this dataset. The x-axis represents the activation time of an alarm, and the y-axis shows the duration (i.e., TimeDelta= StartTime - EndTime) of the corresponding activation. As we can see that most of the alarms deactivated within 20 seconds (it will be more clear in followings sections).

In [22]:

# fig = px.scatter(df, x="StartTime", y="TimeDelta",render_mode="webgl")
# fig.show()

# Box Plot of  TimeDelta for all alarms
This is one of the most important box plots in the whole analysis. It will help us to determine the threshold for TimeDelta. If the duration between activation and deactivation (i.e., TimeDelta = Deactivation - Activation) is less than the threshold then we will not transmit such alarm to the historian. From the following box plot, we can see that the first quartile (q1) is equal to 16 seconds which means if we set the threshold constant equals to 16 seconds we will directly reduce 25% of the communications between the DCS systems and Historian server. Similarly, if we set threshold constant to 29 seconds (i.e., q2 or median value) the 50 % of the storage & and communication will be reduced.  
**Questions:**
* We need more data to determine the threshold constant value. Can I get more data of the same plant? For example, 1 year or  6 months.
* Can I choose threshold value equals to 29 seconds?

**Note: If hover the mouse over any graph it will show the values.**

In [12]:
# fig = px.box(df, y="TimeDelta")
# fig.update_yaxes(range=[0, 100])
# fig.show()

# SourceName Analysis

From the first histograms, we can see that "47TI931" triggered the most number of alarms. Additionally, all the alarms are related to “IOP” condition. 

**Questions** 
* Why so many communications problems occur? Is it normal?


In [10]:
# fig = px.histogram(df, x = "SourceName")
# fig.update_layout(title="Number of times each SourceName triggered an alarm.")
# fig.show()

# fig = px.histogram(df, x = "SourceName", color="Condition",  barmode='group')
# fig.update_layout(title="Number of times a condition occured for a SourceName.")
# fig.show()

# fig = px.box(df, x= "SourceName", y="TimeDelta")
# fig.update_yaxes(range=[0, 200])
# fig.update_layout(title="Box Plot of TimeDelats (Deactivaion - Activation) for each SourceName")
# fig.show()


# Conditions Analysis

As we can see from the following graphs, the most frequent alarms are related to communication problems. For instance, IOP condition occurred more 191 thousand time. Additionally, from the box plot, we can see that that durations (i.e., timedelta = deactivation time - activation time)  of IOP alarms are shorter.

**Conclusion:** IOP communications can discard easily as it just represent only the communication issue between the field device and the DCS system. So if we discard such alarms we reduce the communication between the DCS system and historian server to a few hundreds instead of thousands. 

**Questions:**
* Why the Vel+ and Vel- conditions are lesser than compared to other conditions? Because they were the most in 1-day data (old data). Is it normal?
* Whether my conclusion is correct or not? 


In [1]:
# fig = px.histogram(df, x = "Condition")
# fig.show()


# fig = px.box(df, x= "Condition", y="TimeDelta")
# fig.update_yaxes(range=[0, 200])
# fig.show()

### Condtions Occuring on each day of Week

In [21]:
# fig = px.histogram(df, x="WeekDay", color="Condition",  barmode='group')
# fig.show()

# Chattering Processing and Algorithms

In [3]:
sources = {} # contains each unique sourcename as key and corresponding alarms in a list
for sname in df["SourceName"].unique():
    sources[sname] =  []
temp_dict = None
for i in range(df.shape[0]):
    temp_dict = {}
    for key in df.columns:
        temp_dict[key] = df[key][i]
    sources[df["SourceName"][i]].append(temp_dict)

### Chattering Algorithm

In [10]:
def findChatterings(alarms, chattering_timedelta_threshold=60.0, chattering_count_threshold=3):
    chattering = {}
    
    alarms = [alarm for alarm in sorted(alarms, key=lambda arg: arg["StartTime"], reverse=False)]    
    count = 0
    
    for i in range(len(alarms)):
        t_prev = alarms[i]["StartTime"]
        count = 0
        for j in range(i+1, len(alarms)):
            t_next = alarms[j]["StartTime"]    
            
            if timedelta.total_seconds(t_next - t_prev) > chattering_timedelta_threshold:
                break
                
            count += 1
#             print(time_delta, "count ++ ", count, t_prev, t_next)
        
#         if count > chattering_count_threshold:
        chattering[t_prev] = {"own_index": i, "count": count}
            
    return chattering

In [ ]:
chats = findChatterings(sources.k)
chats = [v["count"] for v in chats.values()]
trace = go.Histogram(x=chats)
fig.show(data=[trace])

#### Chattering Histograms 
If an alarm from the same source is triggered 3 times or less then such situation of chattering is considered normal.  However, from the following the histograms, we can see that thousands of alarms are triggered more than thrice in most of the sensors. For instance, consider "47TI931A", it chatters 6 times in a minute more than 31 thousand times. 

**Note:  x-axis represents the number of times an alarm chatter in the duration of one minute. The y-axis represents how many times such conditions (i.e., chatters) occur.** 

**Conclusion:** Such conditions (in which an alarm is triggered more than 3 times in a minute) are abnormals. So, if we detect such conditions at the edge it may help to do.... [list the usages].
 
**Questions:** 
* Is my conclusion correct? 
* What are the use cases if we detect chatters in real-time? 
* How too much chattering effect system performance?

In [23]:
# cols = 2
# rows = math.ceil(len(sources.keys())/cols)
# fig = make_subplots(rows=rows, cols=cols)


# t = 0
# snames = list(sources.keys())
# for row in range(1, rows+1):
#     for col in range(1,cols+1):
#         chats = findChatterings(sources[snames[t]])
#         chats = [v["count"] for v in chats.values()]
#         trace = go.Histogram(x=chats,name=snames[t])
#         fig.add_trace(trace,row =row, col=col)
#         t += 1
# fig.show()

# Time Between form one Deactivation to Next Deactivation for whole Data

In this section, I tried to find the time between 1 deactivation to next deactivation. In ideal scenario, there should be 1 alarm per minute. However, when I took the time difference between 1 deactivation to the next activation of any alarm the difference (delta) was mostly under 10 seconds which means that a huge number alarms are generated in 1 minute.

**Note: The x-axis represents the time delta (i.e., the time difference between deactivation and activation of an alarm), while the y-axis represents the frequency (count) of corresponding deltas.**

**Questions:**  
* Are my observations correct? If not please let me know what should I do to cross-check my results
* If my observations are correct then what kind of conclusions I can draw from this graph?


In [42]:
def frequencyOfAlarmsActivated(alarms, timediff=60):
    alarms_by_start_time = [alarm for alarm in sorted(alarms, key=lambda arg: arg["StartTime"], reverse=False)]
    alarms_by_end_time = [alarm for alarm in sorted(alarms, key=lambda arg: arg["EndTime"], reverse=False)]
    freq = []
    max_delta = -1
    temp = 0
    for i in range(len(alarms)):
        t_end = alarms_by_end_time[i]["EndTime"]
        j = 0 
        for j in range(temp,len(alarms)):        
            t_start = alarms_by_start_time[j]["StartTime"]
            delta = timedelta.total_seconds(t_start - t_end)
            
            if delta< 0:
#                 print(delta)
                continue
            else:
                temp = j-1
                if max_delta < delta:
                    max_delta = delta
#                 print(delta)
                freq.append(delta)
                break
#     print("Max seconds :", max_delta)
    return freq

In [24]:
# alarms = []
# # concatenating lists of alarms
# for sname in sources.keys():
#     alarms = alarms + sources[sname] 

# freq = frequencyOfAlarmsActivated(alarms)
# d = {}
# for v in freq:
#     if v < 200:
#         d[v] = 0

# for v in freq:
#     if v < 200:
#         d[v] +=1

# counts = [v for v in d.values()]
# deltas = [k for k in d.keys()]

# fig = px.bar(x=deltas, y = counts)
# fig.show()